In [3]:
import os

import pandas as pd
from pandas import DataFrame, Series
from tqdm.auto import tqdm
from pandarallel import pandarallel

tqdm.pandas()
pandarallel.initialize(progress_bar=False)
# Сброс ограничений на число столбцов
pd.set_option("display.max_columns", 200)

# Сброс ограничений на количество символов в записи
pd.set_option("display.max_colwidth", 200)

INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


Что надо сделать:
- заполнение бюджетного уровня
- адресов заказчиков (найти плохие адреса и тоже их поменять)
- дополнение адресов поставщиков (теже что по инн и ближайшие по времени)
- расшифровка кбк 
- расшифровка адресов 
- 

In [4]:
df = pd.read_csv('../data/raw_data/contract/2014/0.csv', sep='|', dtype='str')

In [5]:
df.columns

Index(['number_contract', 'adress_customer', 'full_name_customer',
       'short_name_customer', 'unique_site_code', 'unique_site_id',
       'id_customer', 'inn_customer', 'kpp_customer', 'code_form_org',
       'okpo_code', 'municipal_code', 'budget_name', 'extrabudget_name',
       'budget_level', 'contract_status', 'notice', 'ikz_code',
       'id_contract_electronic', 'unique_number_plan',
       'method_determinig_supplier', 'date_summarizing', 'date_posting',
       'grouds_single_supplier', 'document_details', 'info_support',
       'find_date_contract', 'date_performance', 'date_contract_registry',
       'date_update_registry', 'date_start_performance',
       'date_end_performance', 'contract_item', 'contract_price',
       'contract_price_nds', 'prepayment_amount', 'performance_security',
       'size_performance_quality', 'warranty_period', 'place_performance',
       'full_name_supplier', 'inn_supplier', 'kpp_supplier',
       'code_okpo_supplier', 'date_registration_supp

## Дополнение уровней бюджета

In [6]:
list_local = ['Муниципальный уровень', 
            'местный бюджет']
list_sub = ['Уровень субъекта РФ', 
            'бюджет субъекта Российской Федерации', 
            'бюджет территориального государственного внебюджетного фонда',
            'бюджет территориального государственного внебюджетного фонда']
list_fed = ['Федеральный уровень', 
            'федеральный бюджет', 
            'бюджет Пенсионного фонда Российской Федерации', 
            'бюджет Федерального фонда обязательного медицинского страхования', 
            'бюджет Фонда социального страхования Российской Федерации']

list_local_2 = ['муниципаль', 'школа', 'детский сад']
list_sub_2 = ['центр занятости населения', 'област', 'республи', 'края', 'край', 'краевое',
              'здравоохране', 'профессиональн']
list_fed_2 = ['войскавая', 'воениз', 'федеральн',
              'следствен', 'пенсион', 'росгран', 'фгбу', 'всероссийс', 
             'университ']

In [7]:
def fillna_budget_level(budget_level:str, full_name_customer:str):
    for list_name, name in zip([list_local, list_sub, list_fed], ['местный', 'субъектовый', 'федеральный']):
        for name_trigger in list_name:
            if name_trigger.lower() in budget_level:
                return name
    
    # если не получилось выделить данные из budget_level попробуем сделать это с full_name_customer
    for list_name, name in zip([list_local_2, list_sub_2, list_fed_2], ['местный', 'субъектовый', 'федеральный']):
        for name_trigger in list_name:
            if name_trigger.lower() in full_name_customer:
                return name
    
    return budget_level

In [8]:
def processing_budget(X: Series):
    budget_level = str(X['budget_level']).lower()
    full_name_customer = str(X['full_name_customer']).lower()
    
    budget_level = fillna_budget_level(budget_level, full_name_customer)
    return budget_level
    

In [9]:
df_test = df.copy()

In [10]:
df_test['budget_level_new'] = df_test.apply(processing_budget, axis=1)

In [11]:
df_test['budget_level_new'].isnull().sum()

0

In [12]:
## проверка на других данных
path_test = '../data/raw_data/contract/2014/'
list_problem = []
for file_name in tqdm(os.listdir(path_test)):
    df_test_now = pd.read_csv(os.path.join(path_test, file_name), sep='|', dtype='str')
    df_test_now['budget_level_new'] = df_test_now.parallel_apply(processing_budget, axis=1)
    if df_test['budget_level_new'].isnull().sum() == 0:
        continue
    else:
        list_problem.append((file_name, df_test['budget_level_new'].isnull().sum()))
    

100%|██████████| 479/479 [26:40<00:00,  3.34s/it]


In [13]:
list_problem

[]